# ECE 670 X1 Mini Project 1

 Sharyar Memon, 1299819
<br>
Alyson Wu, 1399985

## Task 1 - Mosquito Trap Data and Weather Data

<b>Problem Statement</b>

For Task 1, we were tasked in the creation of 3 regression models using Edmonton weather data and Edmonton Mosquito
Trap data. These models were meant to represent a function between the number of observed mosquitoes when considering a
number of weather features.

In total, 3 models were created:
1. A linear regression model to represent the number of observed mosquitoes given a number of weather features. 
2. A linear regression model to represent the number of female mosquitoes given the same weather data.
3. A polynomial model to model the number of female mosquitoes given the same weather data.

### Data Pre-processing

Prior to creation of a linear regression model, both data sets (Edmonton Mosquito Trap Data and Edmonton Weather Data)
had to be concatenated into a singular data set for analysis. During the initial inspection of the data, the following
was observed:
1. Additional logic had to be added to resolve the 'time grid' utilized by the two sets of data. Where the Mosquito
Trap Data was sampled at a frequency of one data point per week, Edmonton Weather data was sampled once every hour.
2. Because Edmonton weather data is an instantaneous measure at the recorded time, some kind of aggregation of the
weather data is desired to better match the Mosquito Trap data, such as the utilization and calculation of a moving
average for certain features in the weather data set (humidity, temp, dew point, wind_dir_10s, wind_speed, health_index)
3. Within both data sets, some features have inconsistent data and therefore was ignored from the analysis.
Columns with limited or no data, such as visibility, cloud cover, humidex, windchill and solar radiation
were identified as unnecessary to our analysis, and was removed. Similarly, rows with missing data points were
also removed from our analysis.
4. For the Edmonton Mosquito Trap data, rows marked with UnID from the IDd column represented bad data and were
removed from the analysis.

<b>Mosquito Data Pre-processing</b>

For the Mosquito Trap data, we inferred that an average of roughly 20 observations were recorded on a weekly basis of
the number of female and male mosquitoes that were caught in the recovered mosquito traps that were placedk in various
locations in Edmonton. At a glance, it is clear that this data set contains data spanning from 1990 to 2020.

The first desired model takes into account all mosquito genders (male and female) and therefore a method was scripted
in order to facilitate ease in isolating parts of the data set we were interested in if we desired (introduction of an
optional parameter to isolate either only females or only males). Additionally, the helper functions provided
convenience in cleaning the data set in the following ways:
1. The provided trap date column was processed as a datetime object for correct typing
2. Unnecessary columns of the mosquito trap data were removed. These columns included the following: 'Comparison Group',
 'Genus', 'Specific Epithet', 'Trap Region', 'Latitude', 'Longitude', 'Location', 'IDd', 'Include'
3. Removal of all rows marked as No or No Data in the Include column - from the data set description, values of No or
No Data are indicative that the data is of bad quality and therefore should not be considered in our analysis.
4. Summing of the number of mosquitoes on a given date given the mosquito data (all data, female only data, male only data)
5. As our constructed model only requires the number of observed mosquitoes, only the summed number of mosquitoes and
trap date is retained in the final data set.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error
import helper_functions
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import cross_val_score
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Loading of the mosquito trap data set
mosquito_df = pd.read_csv('data_files/Mosquito_Trap_Data.csv')

# Using helper functions to clean mosquito data frame with previously discussed operations
mosquito_df_all = helper_functions.clean_mosquito_df(mosquito_df, gender=None) # both male and female combined
mosquito_df_all

Following the initial clean up of just the mosquito trap data set, there are 578 rows of data, which is significantly
lower than the original data set which contained 27263 rows of data. Note that this is still sensible as an average of
20 traps were collected on any given trap date, which reduces the intial row value to approximately 1363 rows of data.
Additionally, a total of 829 rows were discarded as these rows were marked as No or No Data in the Include column,
which signifies bad quality data. Hence, our remaining data set for the mosquito data was found to have 578 rows of
data from the initial data set.

<b>Edmonton Weather Data Pre-processing</b>

The feature set utilized in the prediction of the number of mosquitoes found on a given trap date was taken from the
Edmonton Weather data set. At a first glance of the data set, weather related data was recorded on an hourly time
interval for the time period of 2017-2018.

Because the Edmonton weather data consists of instantaneous recordings of Edmonton weather, it was determined that
further data processing would be desirable to better relate the weather to the weekly recorded mosquito trap data. Note
that in this analysis, we assume that the mosquito traps are collected from various locations on a weekly basis, and as
such the number of mosquitoes that are counted are reflective of a weeks worth of mosquitoes landing in the mosquito
trap. Because this observed mosquito count value is not an instantaneous value, it is appropriate to attempt to
match weather data to the same observed period of time (~ 1 week).

When performing data pre-processing for the weather data set, the following operations were conducted:
1. The provided unixtime column was processed as a datetime object for correct typing
2. Moving averages were calculated for a 7 day moving average for specific weather
features ('pressure_station', 'pressure_sea', 'wind_dir_10s', 'wind_speed', 'relative_humidity', 'dew_point',
'temperature')
3. Unnecessary columns of the weather data were removed. These columns included the following: 'unixtime',
'wind_dir', 'windchill', 'humidex', 'visibility', 'health_index', 'cloud_cover_4', 'cloud_cover_8', 'cloud_cover_10',
'solar_radiation'
4. We also create scaled versions of each feature, and the label to compare the performance of the scaled vs unscaled
models.

In [ ]:
# Load the weather data
weather_df = pd.read_csv('data_files/weather_stats_edmonton.csv')

# Clean weather data and perform preprocessing (performing moving average of 7D)
weather_df = helper_functions.clean_weather_df(weather_df)
weather_df = helper_functions.preprocess_weather(weather_df, ['7D'])

<b>Concatenation of the Two Data Sets</b>

Merging of the two data sets was performed with the intent of matching the trap dates given by the mosquito trap data
set. Note that the range of time values given in the weather data set (2017-2018) was not extensive enough to cover the
entire period covered by the mosquito trap data (1990-2020). As a result, following the merge of the two data sets,
only 41 rows were present in the final merged data set.

**Data Set Limitations**

It is of interest to note that the limited number of samples that we were able to obtain following the merge will
likely impact both training and testing of the generated models. Because the limiting data set is the Edmonton weather
data, it is suggested that more weather data should be located so that more mosquito trap dates can be utilized for
better prediction accuracy. As this was beyond the scope of the Mini Project, retrieval of additional weather data was
not conducted; however, it is noted as a limitation for generated model prediction accuracy.

In [ ]:
merged_df = helper_functions.merge_mosquito_weather_data(mosquito_df_all, weather_df)

# Removal of columns that reflect instantaneous observations as we will lean towards using moving averages
# for the creation of the models
instantaneous_weather_recordings = ['pressure_station', 'pressure_sea', 'wind_dir_10s', 'wind_speed',
                                   'wind_gust', 'relative_humidity', 'dew_point', 'temperature']
merged_df.drop(columns=instantaneous_weather_recordings, inplace=True)

# Display the merged data frame
merged_df

### Regression Models

### 1. Number of Mosquitoes vs. Weather Features (Linear Regression)

The first model of interest is the creation of a linear regression model that predicts the number of mosquitoes
(in the collected mosquito traps) based on observed Edmonton weather features. For this analysis, the total number of
mosquitoes are used.

Note that, due to a limited number of observations (~40 observations), data was not split into a training and testing data set for the linear regression model.

In [ ]:
# Define feature set (weather features) and resulting output (number of mosquitoes)
X = merged_df.drop(columns=['Count'])
Y = merged_df['Count']

A preliminary view of the data set across all features is shown below. We have completed this exercise to gain insight on the feature space to see whether there were any initial patterns that could be viewed across each feature.

In [ ]:
# Plot all features against counts:
fig, axes = plt.subplots(len(X.columns.tolist()), figsize=(8,40))
for i,feature in enumerate(X.columns.tolist()):
    axes[i].scatter(X[feature], Y)
    axes[i].set_ylabel('Count')
    axes[i].set_xlabel(feature)
plt.show()

### Train 3 different models with no scaling/normalization, scaling, normalization

For this exercise, we created three linear regression models using different variations of the original data set (original data, scaled data and normalized data). Following this, the performance of each model was compared, and lastly, feature importance was 
1. Create linear regression models
2. Compare performance
3. Check feature importance in the best model

Standard Scaling:
$$z = \frac{(x - \mu)} {s}$$

For linear regression, 3 models were trained with the regular data (non_scaled_reg), scaled data (pipe_scaled_reg) and normalized data (pipe_normalized_reg). These models were generated using Pipeline to simplify the coding process.

In [ ]:
# 3 Models:
pipe_scaled_reg = Pipeline([('scaler', StandardScaler()), ('linear_regressor_scaled', linear_model.LinearRegression())])
pipe_normalized_reg = Pipeline([('normalizer', Normalizer()), ('linear_regressor_normalized', linear_model.LinearRegression())])
non_scaled_reg = linear_model.LinearRegression() # since all quantities are non-negative

In [ ]:
pipe_scaled_reg.fit(X, Y)
pipe_normalized_reg.fit(X, Y)
non_scaled_reg.fit(X, Y)

predictions = {
    'Raw Regression' : non_scaled_reg.predict(X),
    'Scaled Data' : pipe_scaled_reg.predict(X),
    'Normalized Data': pipe_normalized_reg.predict(X)
}

scores = {
    'Raw Regression': [
        mean_squared_error(Y, predictions['Raw Regression'], squared=False),
        r2_score(Y, predictions['Raw Regression'])
    ],
    'Scaled Data': [
        mean_squared_error(Y, predictions['Scaled Data'], squared=False),
        r2_score(Y, predictions['Scaled Data'])
    ],
    'Normalized Data': [
        mean_squared_error(Y, predictions['Normalized Data'], squared=False),
        r2_score(Y, predictions['Normalized Data'])
    ]
}
pd.DataFrame(scores, index=['MSE', 'R2'])

Evaluation of the best model can be achieved by the analysis of the MSE (mean squared error) and the R2 score. The mean squared error is defined as the average of the square of the errors, which is the difference between the observed value and the predicted value. The difference of these values are then squared and summed. The R2 score the regression score function, or coefficient of determination.

In the determination of the best model, a low MSE is desired, and generally a high R-squared value is desired (with some caveats). Note that R-squared always falls between 0 and 1, where a 0 indicates that the model does not explain the variability of output data around its mean, and and value of 1 indicates that the model is able to explain all variabiility of output data around its mean.

**Best Linear Regression Model:** The linear model trained with normalized data showed the best performance with the lowest MSE value and the highest R2 score.

In the next cells, we perform feature set analysis on this model to determine the coefficients of features for linear regression. This is done to determine with features are more important than other features within the data set and which features contribute the most to the studied response.  

In [ ]:
plt.figure(figsize=(15,8))
plt.bar(X.columns.tolist(), pipe_normalized_reg[1].coef_)
plt.xticks(rotation=90)
plt.title('Coefficients of Features for Linear Regression')

From the coefficients of features for linear regression figure shown above, it is clear that the two features that contribute most are the pressure_station_7D and the pressure_sea_7D. The next features that show some contribution are the relative humidity (7 day moving average) and the temperature 7D. Other features within the feature set show very little contribution in comparison. 

Lastly, we can demonstate the comparisons of predictions vs. the true values of this linear regression model, which is shown below:

In [ ]:
plt.figure(figsize=(15,8))
plt.scatter(X.index, Y)

plt.scatter(X.index, predictions['Raw Regression'], marker='s')
plt.scatter(X.index, predictions['Scaled Data'], marker='x')
plt.scatter(X.index, predictions['Normalized Data'])

plt.legend(['True', 'Raw Regression', 'Scaled', 'Normalized'])
plt.title('Comparison of Predictions and True values')
plt.xlabel('Dates')
plt.ylabel('Mosquito Count')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('Mosquito-Predictions.png')

### Insights about predictions:

* We can see that there are data points missing for the winter months in the mosquito data set.
* That makes sense given we do not see mosquitoes in winter months however we can not assume a number without more
infomation provided to us by the original collectors of the data. This impairs our model to some extent.
* We can also see that given the limited number of observations, the various models can make approximate predictions.
* We can further see that there is seasonality to the count of mosquitoes however due to assignment limitations, we can
only use features that originate in the weather data set. As such that was not included in the feature set.

# Task 1 - Part 2

For the second part of task 1, we are working on female mosquitoes and trying to predict their number by using a linear regression model and polynomial regression model. This analysis is similar to the previous analysis using the full data set (both females vs. males), except we have decided to change the feature set that will be analyzed. For the following analysis, we will use the instantaneous weather features and 14d moving averages as features as a starting point for the creation of the model.

Following this analysis, in the last part of task 1, we will then create model using polynomial features and see if they provide a better model. These models are analyzed separately for convenience.

Again, we must first begin by loading the mosquito trap data set, and performing isolation to target only the female population of mosquitoes. Note that this is achieved using methods that are defined in a helper_functions script that is external to the Jupyter notebook. 

In [ ]:
# Loading of the mosquito trap data set
mosquito_df = pd.read_csv('data_files/Mosquito_Trap_Data.csv')

X_train, X_test, Y_train, Y_test = train_test_split( X , Y , test_size = 0.2, random_state = 0)

# Using helper functions to clean mosquito data frame with previously discussed operations
mosquito_df_female = helper_functions.clean_mosquito_df(mosquito_df, gender='Female') # female only data

In [ ]:
mosquito_df_female

Similar operations are performed for the weather data set, except we instead are interested in performing an analysis using a 14 moving day average instead of the previously examined 7 day moving average. Again, generation and cleaning of the data is achieved using helper function methods that can be found in the external Python script.

In [ ]:
# Load the weather data
weather_df = pd.read_csv('data_files/weather_stats_edmonton.csv')

# Clean weather data and perform preprocessing (performing moving average of 14D)
weather_df = helper_functions.clean_weather_df(weather_df)
weather_df = helper_functions.preprocess_weather(weather_df, ['14D'])

In [ ]:
merged_df = helper_functions.merge_mosquito_weather_data(mosquito_df_female, weather_df)

# Display the merged data frame
merged_df

Again, similar to previous analysis, the dataframe is then split into the feature set (X) and the output response (Y). The feature set is also plotted against the response to facilitate information gathering to see whether there were evident patterns in the feature set. 

Note that the splitting of the data set is shown below as a placeholder for future analysis later in the notebook.

In [ ]:
X = merged_df.drop(columns='Count')
Y = merged_df['Count']

# Splitting into training set and testing set
X_train, X_test, Y_train, Y_test = train_test_split( X , Y , test_size = 0.2, random_state = 0)

In [ ]:
# Plot all features against counts:

fig, axes = plt.subplots(len(X.columns.tolist()), figsize=(8,40))
for i,feature in enumerate(X.columns.tolist()):
    axes[i].scatter(X[feature], Y)
    axes[i].set_ylabel('Count')
    axes[i].set_xlabel(feature)
plt.show()

### Train 3 different models with no scaling/normalization, scaling, normalization

Similar to the work conducted previously when creating a linear model across all mosquitoes, the same analysis is perofrmed for the female only data set. Again, we created models for each variation of the feature data set which included regular, non-scaled and normalized data. Performance was compared between each model to determine which model was the best, followed by the feature importance analyis in the best model.

1. Create linear regression models using different variants of the female-only data (scaled, non-scaled, normalized)
2. Compare performance across models

Standard Scaling:
$$z = \frac{(x - \mu)} {s}$$

### 2. Linear Regression Model - Female Mosquitoes vs. Weather Features

In [ ]:
# 3 Models:
pipe_scaled_reg = Pipeline([('scaler', StandardScaler()), ('linear_regressor_scaled', linear_model.LinearRegression())])
pipe_normalized_reg = Pipeline([('normalizer', Normalizer()), ('linear_regressor_normalized', linear_model.LinearRegression())])
non_scaled_reg = linear_model.LinearRegression() # since all quantities are non-negative

In [ ]:
pipe_scaled_reg.fit(X, Y)
pipe_normalized_reg.fit(X, Y)
non_scaled_reg.fit(X, Y)

predictions = {
    'Raw Regression' : non_scaled_reg.predict(X),
    'Scaled Data' : pipe_scaled_reg.predict(X),
    'Normalized Data': pipe_normalized_reg.predict(X)
}

scores = {
    'Raw Regression': [
        mean_squared_error(Y, predictions['Raw Regression'], squared=False),
        r2_score(Y, predictions['Raw Regression'])
    ],
    'Scaled Data': [
        mean_squared_error(Y, predictions['Scaled Data'], squared=False),
        r2_score(Y, predictions['Scaled Data'])
    ],
    'Normalized Data': [
        mean_squared_error(Y, predictions['Normalized Data'], squared=False),
        r2_score(Y, predictions['Normalized Data'])
    ]
}

pd.DataFrame(scores, index=['MSE', 'R2'])

In the determination of the best model, a low MSE is desired, and generally a high R-squared value is desired, as was explained previously.

**Best Linear Regression Model For Female Mosquito Data:** The linear model trained with normalized data showed the best performance with the lowest MSE value and the highest R2 score.

### 3. Polynomial Features Model - Female Mosquitoes vs. Weather Features

To aid us in the creation of a polynomial model for the female mosquito data, an F-test should be conducted to determine the best degree of the polynomial model to generate. This is illustrated in the next section.

<b>Feature Selection</b>

For regression analysis, an F-test should be performed to ascertain the most significant features of the data set.
This is done to remove features that do not contribute to the model's performance and determine which features are the
most significant prior to proceeding to our final analysis. In particular, the F-test estimates the degree to
which there is a linear dependency between two random variables when comparing their variances (in our case, the feature
set, and the output label), and the p value allows us to determine significance through the validation or rejection of
the null hypothesis. The null hypothesis assumes that there is no relationship between a given input feature and an
output.

In [ ]:
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

In [ ]:
F_test,p_value = f_regression(X_poly,Y)
feature_f_test_df = pd.DataFrame({'feature':poly_features.get_feature_names(),
              'F_test':F_test,'p_value':p_value})

In [ ]:
feature_f_test_df

In [ ]:
# Only contains features that p-values < 0.05
best_features_idx = feature_f_test_df[feature_f_test_df['p_value'] < 0.05].index

X_best = SelectKBest(f_regression,k=5).fit_transform(X_poly, Y)

Following the F-test and p value analysis, selection of the best features is performed based on p-value.
Some of these features are also used to visualize the relationship between the female mosquito counts and the feature. These figures are shown below: 

In [ ]:
plt.scatter(X_poly[:, best_features_idx[0]], Y)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_poly[:, best_features_idx[0]], Y, X_poly[:, best_features_idx[4]])

In [ ]:
poly_pipeline = Pipeline([('Polynomial Features', PolynomialFeatures()),
                          ('Linear Regression', linear_model.LinearRegression())])

In [ ]:
feature_f_test_df.sort_values(by='p_value')

For easier analysis and to compare the degrees and performance, the MSE and standard deviation values were computed up the the 5th degree. This is performed so that we can select the optimum degree for which to generate our polynomial model. In this analysis, training data is used to train our models, and a k-fold corss validation is performed to better determine the accuracy of each generated polynomial model. Relevant code and the results are demonstrated below:

In [ ]:
# Comparing Degrees and performance:
def compute_mse_cv(X, Y, n_degree=5):
    mse = []
    std = []
    for degree in range(1,n_degree):
        poly_features = PolynomialFeatures(degree=degree)
        sklreg = linear_model.LinearRegression()
        pipeline = Pipeline([("polynomial_features", poly_features),
                             ("linear_regression", sklreg)])
        pipeline.fit(X, Y)

        # Evaluate the models using cross validation
        scores = cross_val_score(pipeline, X, Y,
                                 scoring="neg_mean_squared_error",
                                 cv=10, error_score=np.nan)

        mse.append(-scores.mean())
        std.append(scores.std())
    return mse,std

In [ ]:
mse,std = compute_mse_cv(X_train,Y_train)

In [ ]:
plt.plot(range(1,len(mse)+1),mse, label='MSE')
plt.plot(range(1,len(std)+1),std, label='std')
plt.legend();

Again, in the selection of the best perfoming model, the model with the lowest mean squared error is desired, as well as a model that can account for a lower standard deviation. The standard deviation metric is closely related to the variance. A higher variance may be indicative that the model not stable across different training sets (following the cross validation), which then may indicate that there is a higer risk of overfitting.

For this case scenario, we've chosen a degree of 2 for the polynomial features. This degree is present at the bend of the curve shown above and has a relatively low MSE (in comparison to the higher degree polynomial models) and a low standard deviation value (in comparison to the higher degree polynomial models). Note that a degree of 1 was not chosen as that would be reflective of a linear regression model and at a degree = 1 a polynomial model would be pointless.

In [ ]:
# 3 Models:
poly_pipeline = Pipeline([('Polynomial Features', PolynomialFeatures(degree=2)),
                          ('Linear Regression', linear_model.LinearRegression())])
raw_regressor = linear_model.LinearRegression()

kbest_pipe = Pipeline([('Select K Best', SelectKBest(f_regression, k=3)), ('Linear Regressor', linear_model.LinearRegression())])

### Comparison of Female Mosquito vs. Weather Features Models

In [ ]:
poly_pipeline.fit(X, Y)
raw_regressor.fit(X, Y)
kbest_pipe.fit(X_poly, Y)

predictions = {
    'Polynomial Features - 2 Degrees' : poly_pipeline.predict(X),
    'Linear Regression' : raw_regressor.predict(X),
    'K Best Pipeline - 3 Features': kbest_pipe.predict(X_poly)
}

scores_pipe = {
    'Polynomial Features - 2 Degrees': [
        mean_squared_error(Y, predictions['Polynomial Features - 2 Degrees'], squared=False),
        r2_score(Y, predictions['Polynomial Features - 2 Degrees'])
    ],
    'Linear Regression': [
        mean_squared_error(Y, predictions['Linear Regression'], squared=False),
        r2_score(Y, predictions['Linear Regression'])
    ],
    'K Best Pipeline - 3 Features': [
        mean_squared_error(Y, predictions['K Best Pipeline - 3 Features'], squared=False),
        r2_score(Y, predictions['K Best Pipeline - 3 Features'])
    ]
}

pd.DataFrame(scores_pipe, index=['MSE', 'R2'])


In comparing the three generated models, it is clear that the polynomial features has the least MSE (mean squared error), but a high R2 score of 100% which indicates that the polynomial features model was 100% accurate. This leads to potential that the model was overfitted since the R2 score value shows that the model can account for 100% of the data set's variance. This problem may be rectified by a larger sample set - due to our data set being limited, it is fully plausible that the data does not accurately reflect what the total variance of the data set could be if we were given more than 40 observations to work with.

The linear regression model had a MSE of ~631 which is a very large error considering that the number of mosquitoes in the response data falls between values of 28-2959. Similarly, the k-best model has an even higher MSE value of ~761. Generally a high value of R2 is desired for best performance. Therefore, the linear regression model outperforms the k-best pipeline model in this respect.